In [ ]:
import openpyxl
from openpyxl.cell import MergedCell
import xml.etree.ElementTree as ET
from xml.dom import minidom

# --- Helper Function to get merged cell value ---
def get_cell_value(sheet, row, col): # Correct argument names are 'row' and 'col'
    """
    Retrieves the value of a cell, handling merged cells.
    """
    cell = sheet.cell(row=row, column=col)
    if isinstance(cell, MergedCell):
        for merged_range in sheet.merged_cells.ranges:
            if cell.coordinate in merged_range:
                # Return the value of the top-left cell in the merged range
                return sheet.cell(row=merged_range.min_row, column=merged_range.min_col).value
    return cell.value

# --- Main Script ---

def convert_excel_to_xml(excel_file, sheet_name, output_xml_file):
    # Load the workbook and select the sheet
    try:
        wb = openpyxl.load_workbook(excel_file)
        sheet = wb[sheet_name]
    except FileNotFoundError:
        print(f"Error: {excel_file} not found.")
        return
    except KeyError:
        print(f"Error: Sheet '{sheet_name}' not found in the workbook.")
        return

    # Create the root element of the XML
    report = ET.Element("FileInformation")

    # --- Extract specific single-mapped cells and add to XML ---
    
    #  FileNumber (Merged cell A1:D1, access using A1 coordinates)
    # Changed 'column=1' to 'col=1'
    report_title = get_cell_value(sheet, row=4, col=3) 
    if report_title:
        ET.SubElement(report, "FileNumber").text = str(report_title)

    #  FileType (B2)
    # Changed 'column=2' to 'col=2'
    project_name = get_cell_value(sheet, row=3, col=3) 
    if project_name:
        ET.SubElement(report, "FileType").text = str("Test report")

    # Date (D2)
    # Changed 'column=4' to 'col=4'
    report_date = get_cell_value(sheet, row=3, col=5) 
    if report_date:
        # Format date if necessary, here assumed as string/date object compatible with str()
        ET.SubElement(report, "Date").text = str(report_date)

    # Manager (B3)
    # Changed 'column=2' to 'col=2'
    manager = get_cell_value(sheet, row=4, col=3)
    if manager:
        ET.SubElement(report, "Manager").text = str(manager)

    # Status (D3)
    # Changed 'column=4' to 'col=4'
    status = get_cell_value(sheet, row=4, col=5)
    if status:
        ET.SubElement(report, "Status").text = str(status)

    # Summary (Merged cell B4:D4)
    # Changed 'column=2' to 'col=2'
    summary = get_cell_value(sheet, row=5, col=3)
    if summary:
        ET.SubElement(report, "Summary").text = str(summary)
        
    # --- Generate the XML string ---
    
    # Helper to make the XML output readable (pretty print)
    xml_string = ET.tostring(report, encoding='utf-8')
    pretty_xml = minidom.parseString(xml_string).toprettyxml(indent="  ")

    # Write the XML to a file
    with open(output_xml_file, "w", encoding="utf-8") as f:
        f.write(pretty_xml)

    print(f"Successfully created XML file: {output_xml_file}")


# --- Run the conversion ---
if __name__ == "__main__":
    EXCEL_FILE_PATH = "sample.xlsx"
    SHEET_NAME = "Sheet1"
    OUTPUT_XML_PATH = "output_report.xml"
    
    convert_excel_to_xml(EXCEL_FILE_PATH, SHEET_NAME, OUTPUT_XML_PATH)
